In [3]:
import os

import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, pipeline, LlamaForCausalLM #AutoModelForCausalLM, 

In [4]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [6]:
! ls -alh /var/model

total 63G
drwxrwxr-x 5 1001 1001 4.0K Nov 13 19:09 .
drwxr-xr-x 1 root root 4.0K Nov 14 15:49 ..
-rw-r--r-- 1 1001 1001 6.1K Nov 13 19:09 .DS_Store
-rw-r--r-- 1 1001 1001 4.0K Nov 13 19:09 ._.DS_Store
drwxr-xr-x 2 root root 4.0K Nov 11 02:21 .ipynb_checkpoints
-rw-rw-r-- 1 1001 1001 3.7K Nov  6 17:14 README.md
drwxr-xr-x 2 root root 4.0K Nov 11 02:56 checkpoints
-rw-rw-r-- 1 1001 1001  638 Nov  6 17:14 config.json
-rw-rw-r-- 1 1001 1001  116 Nov  6 17:14 generation_config.json
-rw-r--r-- 1 1001 1001  984 Nov  7 18:12 model.bak
-rw-r--r-- 1 1001 1001  984 Nov  6 21:02 model.yaml
drwxr-xr-x 3 root root 4.0K Nov 11 03:06 output
-rw-rw-r-- 1 1001 1001 9.2G Nov  6 17:28 pytorch_model-00001-of-00007.bin
-rw-rw-r-- 1 1001 1001 9.1G Nov  6 17:28 pytorch_model-00002-of-00007.bin
-rw-rw-r-- 1 1001 1001 9.1G Nov  6 17:28 pytorch_model-00003-of-00007.bin
-rw-rw-r-- 1 1001 1001 9.1G Nov  6 17:28 pytorch_model-00004-of-00007.bin
-rw-rw-r-- 1 1001 1001 9.1G Nov  6 17:28 pytorch_model-00005-of-00007.b

In [7]:
os.chdir( "/var/model" )

In [8]:
model_id = "Phind/Phind-CodeLlama-34B-v2"

# bnb_8bit_config = BitsAndBytesConfig(
#     load_in_8bit=True,
#     llm_int8_threshold = 6.0,
#     bnb_8bit_compute_dtype=torch.bfloat16
# )
bnb_4bit_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    use_flash_attention_2=True
)

tokenizer = AutoTokenizer.from_pretrained( ".", use_fast=True )
model_4bit = LlamaForCausalLM.from_pretrained( ".", quantization_config=bnb_4bit_config, device_map="auto", low_cpu_mem_usage=True )  
# model_8bit = LlamaForCausalLM.from_pretrained( ".", quantization_config=bnb_8bit_config, device_map="auto", low_cpu_mem_usage=True )


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [9]:
text_generator_4bit = pipeline(
    "text-generation", 
    model=model_4bit, 
    tokenizer=tokenizer,
    torch_dtype=torch.float16, 
    device_map="auto"
)
# text_generator_4bit
# text_generator_8bit = pipeline(
#     "text-generation", 
#     model=model_8bit, 
#     tokenizer=tokenizer,
#     torch_dtype=torch.float16, 
#     device_map="auto"
# )
# text_generator_8bit

In [10]:
model_4bit.hf_device_map
# model_8bit.hf_device_map

{'model.embed_tokens': 0,
 'model.layers.0': 0,
 'model.layers.1': 0,
 'model.layers.2': 0,
 'model.layers.3': 0,
 'model.layers.4': 0,
 'model.layers.5': 0,
 'model.layers.6': 0,
 'model.layers.7': 0,
 'model.layers.8': 0,
 'model.layers.9': 0,
 'model.layers.10': 0,
 'model.layers.11': 0,
 'model.layers.12': 0,
 'model.layers.13': 0,
 'model.layers.14': 0,
 'model.layers.15': 0,
 'model.layers.16': 0,
 'model.layers.17': 0,
 'model.layers.18': 0,
 'model.layers.19': 0,
 'model.layers.20': 0,
 'model.layers.21': 1,
 'model.layers.22': 1,
 'model.layers.23': 1,
 'model.layers.24': 1,
 'model.layers.25': 1,
 'model.layers.26': 1,
 'model.layers.27': 1,
 'model.layers.28': 1,
 'model.layers.29': 1,
 'model.layers.30': 1,
 'model.layers.31': 1,
 'model.layers.32': 1,
 'model.layers.33': 1,
 'model.layers.34': 1,
 'model.layers.35': 1,
 'model.layers.36': 1,
 'model.layers.37': 1,
 'model.layers.38': 1,
 'model.layers.39': 1,
 'model.layers.40': 1,
 'model.layers.41': 1,
 'model.layers.42'

## 13 tokens/s w/ 4-bit, before peft fine tuning
## 7 tokens/s w/ 8-bit, before peft fine tuning

In [11]:
%%time

text = "def get_haversine("
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model_4bit.generate(**inputs, max_new_tokens=400)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


def get_haversine(lat1, lon1, lat2, lon2):
    import math

    # convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    # 6371 km is the earth's radius
    km = 6371 * c
    return km

def get_distance_matrix(locations):
    import numpy as np

    n = len(locations)
    matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(i+1, n):
            lat1, lon1 = locations[i]
            lat2, lon2 = locations[j]
            d = get_haversine(lat1, lon1, lat2, lon2)
            matrix[i, j] = d
            matrix[j, i] = d

    return matrix

locations = [(40.7128, -74.0060), (34.0522, -118.2437), (51.5074, -0.1278)]
matrix = get_distance_matrix(locations)
print(matrix)
CPU times: user 28.9 s, sys: 526 ms, tota

In [13]:
# %%time
prompt= """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a detailed description for the following product: Corelogic Smooth Mouse, belonging to category: Optical Mouse

### Response:"""
# input_ids = tokenizer( prompt, return_tensors="pt" ).input_ids

device = "cuda:0"
input_ids = tokenizer( prompt, return_tensors="pt" ).to( device ).input_ids

generation_output = model_4bit.generate(
    input_ids=input_ids, max_new_tokens=128
)

print(tokenizer.decode(generation_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Create a detailed description for the following product: Corelogic Smooth Mouse, belonging to category: Optical Mouse

### Response:
The Corelogic Smooth Mouse is an optical mouse designed for seamless navigation and precision control. This sleek and compact gadget belongs to the category of optical mice, which use light sensors to detect movement and clicks. It is an ideal choice for users who require a high-quality mouse for their computer or laptop, offering both comfort and functionality.

Key Features:

1. Ergonomic Design: The Corelogic Smooth Mouse features a slim and lightweight design, ensuring a comfortable grip for extended use. The mouse is designed to fit perfectly in


In [12]:
type( model_4bit )

transformers.models.llama.modeling_llama.LlamaForCausalLM

In [13]:
del model_4bit

In [14]:
import gc
gc.collect()

1496

In [8]:
# From https://www.databricks.com/blog/efficient-fine-tuning-lora-guide-llms
import pandas as pd
from datasets import load_dataset
from datasets import Dataset

#Load the dataset from the HuggingFace Hub
rd_ds = load_dataset( "xiyuez/red-dot-design-award-product-description" )

#Convert to pandas dataframe for convenient processing
rd_df = pd.DataFrame( rd_ds[ 'train' ] )

#Combine the two attributes into an instruction string
rd_df[ 'instruction' ] = 'Create a detailed description for the following product: ' + rd_df[ 'product' ] + ', belonging to category: ' + rd_df[ 'category' ]

rd_df = rd_df[ [ 'instruction', 'description' ] ]

#Get a 5000 sample subset for fine-tuning purposes
rd_df_sample = rd_df.sample( n=5000, random_state=42 )

#Define template and format data into the template for supervised fine-tuning
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:

{}

### Response:\n"""

rd_df_sample[ 'prompt' ] = rd_df_sample[ "instruction" ].apply( lambda x: template.format( x ) )
rd_df_sample.rename( columns={ 'description': 'response' }, inplace=True )
rd_df_sample[ 'response' ] = rd_df_sample[ 'response' ] + "\n### End"
rd_df_sample = rd_df_sample[ [ 'prompt', 'response' ] ]

rd_df_sample[ 'text' ] = rd_df_sample[ "prompt" ] + rd_df_sample[ "response" ]
rd_df_sample.drop( columns=[ 'prompt', 'response' ], inplace=True )